In [ ]:
!pip install transformers
!pip install ohmeow-blurr -q
!pip install bert-score -q

In [46]:
#reference: https://towardsdatascience.com/fine-tuning-the-bart-large-model-for-text-summarization-3c69e4c04582
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import fastai.text.all
import transformers
import blurr.text.data.all
import blurr.text.modeling.all


In [47]:
df = pd.read_csv("data_preprocessed.csv")
df.head()

,Unnamed: 0,Intent2Use_Open,Intentnotuse_Open,decontracted_Intent2Use_Open,decontracted_Intentnotuse_Open
0,0,"Self care during work and before and after (deep breathing, positive motivation, olfactory). Sa",.,"Self care during work and before and after (deep breathing, positive motivation, olfactory). Sa",.
1,1,In the applications on mechanical restraints and in management of aggressive behaviors,.,In the applications on mechanical restraints and in management of aggressive behaviors,.
2,2,Apply same type of restraint to wrists/hands as to foot. Adjust tightness so not a chooking hazard,.,Apply same type of restraint to wrists/hands as to foot. Adjust tightness so not a chooking hazard,.
3,3,Properly putting restraints on i.e. feet together closer,.,Properly putting restraints on i.e. feet together closer,.
4,4,1) Completing more casual client debriefs as oppose to just seeing it as a task to check off 2) More motivated to implement self-care strategies,.,1) Completing more casual client debriefs as oppose to just seeing it as a task to check off 2) More motivated to implement self-care strategies,.


In [48]:

# group the responses: group size = 10 for intent and group size = 100 for not intent
from transformers import pipeline

responses_intent = [] # a list of strings, each string contains 10 reponses
i=0
for j in range (50):
  text = ''
  for m in range (10):
   text += df['decontracted_Intent2Use_Open'][i]
   i +=1
  
  responses_intent.append(text)

text = ''
for i in range (500,505):
  text += df['decontracted_Intent2Use_Open'][i]
responses_intent.append(text)
print(f"length of responses_intent: {len(responses_intent)}")

responses_not_intent = []# a list of strings, each string contains 100 responses

i=0
for j in range (5):
  text = ''
  for m in range (100):
   text += df['decontracted_Intentnotuse_Open'][i]
   i +=1
  
  responses_not_intent.append(text)

text = ''
for i in range (500,505):
  text += df['decontracted_Intentnotuse_Open'][i]
responses_not_intent.append(text)
print(f"length of responses_not_intent: {len(responses_not_intent)}")


length of responses_intent: 51
length of responses_not_intent: 6


In [58]:
# try doing zero shot with bart-base
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
print("*********************** Summaries for answers that intent to use the training**************************")
for response in responses_intent:
  print(summarizer(response, max_length=50, min_length=10, do_sample=False))
print("*********************** Summaries for answers that do not intent to use the training**************************")
for response in responses_not_intent:
  print(summarizer(response, max_length=50, min_length=10, do_sample=False))

*********************** Summaries for answers that intent to use the training**************************
[{'summary_text': 'Service user educator present for a different perspective. Self care during work and before and after (deep breathing, positive motivation, olfactory).'}]
[{'summary_text': '1) Debrief meetings right after restraint event/Code white 2) Self-care - create a plan (long/short term) Retraining events at work; setting up restraint work to improve flow at work.'}]
[{'summary_text': 'Incorporate more use of 5- man physical restraints. Practice applying restraints to stretcher, be more active in applying restraints in a restraint event.'}]
[{'summary_text': 'It was an excellent refresher Help me with restrainte vents, confidence in [illegible] 1-2 person events During the application of restraints, debriefing patients, management of violent patient; use hand control.'}]
[{'summary_text': 'I feel more confident with applying restraints and also de-escalation strategies. Thi